<a href="https://colab.research.google.com/github/sonmh79/VRPPD-with-robots/blob/main/R_VRPPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import pandas as pd
import time

In [83]:
demand_data = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='demand',index_col=0)
d_data = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='OD_travel_time',index_col=0)
e3 = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='optimal_solution')

In [84]:
demand_data

,Pickup demand,Delivery demand
Customer ID,,
1,0,1
2,2,0
3,1,0
4,0,1
5,0,1
6,1,0
7,1,0
8,0,1
9,2,0


In [85]:
d_data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00,65.69,85.69,219.38,244.38,253.58,294.72,334.72,283.92,262.78,293.12,358.12,302.32
1,65.69,0.00,64.55,158.24,183.24,192.44,233.58,273.58,222.78,201.64,231.98,296.98,241.18
2,85.69,64.55,0.00,218.24,243.24,252.44,293.58,333.58,282.78,261.64,291.98,356.98,301.18
3,219.38,158.24,218.24,0.00,59.55,137.10,178.24,218.24,167.44,146.30,176.64,241.64,185.84
4,244.38,183.24,243.24,59.55,0.00,162.10,203.24,243.24,192.44,171.30,201.64,266.64,210.84
5,253.58,192.44,252.44,137.10,162.10,0.00,125.69,84.55,183.24,162.10,192.44,257.44,201.64
6,294.72,233.58,293.58,178.24,203.24,125.69,0.00,125.69,224.38,203.24,233.58,298.58,242.78
7,334.72,273.58,333.58,218.24,243.24,84.55,125.69,0.00,264.38,243.24,273.58,338.58,282.78
8,283.92,222.78,282.78,167.44,192.44,183.24,224.38,264.38,0.00,65.69,204.38,269.38,213.58
9,262.78,201.64,261.64,146.30,171.30,162.10,203.24,243.24,65.69,0.00,183.24,248.24,192.44


In [88]:
e3

,# of robots,# of trips,Trip composition
0,1,1.0,{0 -> 2 -> 1 -> 0}
1,NaN,2.0,{0 -> 4 -> 10 -> 11 -> 5 -> 7-> 6-> 0}
2,2,1.0,{0 -> 12 -> 9 -> 8 -> 3 -> 0}
3,Solution value,NaN,2523.45 sec (about 39% more)


In [86]:
class Rvrp:
    def __init__(self, demand_data, d_data, floor, n_robots, capacity):
        self.data = demand_data
        self.d_data = d_data
        self.f = floor
        self.r = n_robots
        self.robots = []
        self._divide_floor()
        self.capacity = capacity

    def _divide_floor(self):

        """Initial Routes Construct"""

        r = []
        for i in range(1, self.f + 1):
            r.append(i)
            if len(r) == round(self.f / self.r):
                self.robots.append(r)
                r = []
        if r:
            self.robots.append(r)

    def _is_delivery(self, node):
        return self.data.loc[node]["Delivery demand"] != 0

    def _separate(self, robot):

        """Separate Delivery with Pickup"""

        d, p = [], []
        for node in robot:
            if self._is_delivery(node):
                d.append(node)
            else:
                p.append(node)
        return [d, p]

    def _get_demand(self, node):

        """Get Demand of Node"""

        if node is None:
            return 0

        if self._is_delivery(node):
            return self.data.loc[node]["Delivery demand"]
        else:
            return self.data.loc[node]["Pickup demand"]

    def cal_cost(self, robots):

        """Calculate Total Cost"""

        cost = 0
        for robot in robots:
            for i in range(len(robot) - 1):
                cost += self.d_data.loc[robot[i]][robot[i + 1]]
        print("Total Travel Time: ", cost)

    def make_trip(self):

        """Make Trip Of Robot"""

        trips = []
        for robot in self.robots:
            trip = [0]
            bot = self._separate(robot).copy()
            dnodes, pnodes = bot[0], bot[1]
            while dnodes and pnodes:

                """1. Get First Node from Delivery and Pickup"""

                dnode = dnodes.pop(0)
                pnode = pnodes.pop(0)
                dnode_demand, pnode_demand = self._get_demand(dnode), self._get_demand(
                    pnode
                )

                """ 1-1. Pickup Node's Demand Affect to Capacity"""

                if dnode_demand > pnode_demand:
                    dc, pc = dnode_demand, pnode_demand
                else:
                    dc = pc = pnode_demand

                trip.append(dnode)
                trip.append(pnode)

                """ 2. Search Delivery Nodes """

                stack = 0
                for i in range(len(dnodes)):
                    curindex = i - stack
                    curdemand = self._get_demand(dnodes[curindex])
                    if dc + curdemand <= self.capacity:
                        dc += curdemand
                        trip.append(dnodes.pop(curindex))
                        stack += 1
                    if dc == self.capacity:
                        break

                """ 3. Search Pickup Nodes  """

                stack = 0
                for i in range(len(pnodes)):
                    curindex = i - stack
                    curdemand = self._get_demand(pnodes[curindex])
                    if pc + curdemand <= self.capacity:
                        pc += curdemand
                        trip.append(pnodes.pop(i))
                        stack += 1
                    if pc == self.capacity:
                        break

                trip.append(0)

            """ 4. Search Rest Nodes (Only Delivery or Pickup) """

            while dnodes:
                dc = 0
                stack = 0
                for n in dnodes:
                    curdemand = self._get_demand(n)
                    if dc + curdemand <= self.capacity:
                        dc += curdemand
                        trip.append(n)
                        dnodes.remove(n)

                    if dc == self.capacity:
                        trip.append(0)
                        break

            while pnodes:
                pc = 0
                for n in pnodes:
                    curdemand = self._get_demand(n)
                    if pc + curdemand <= self.capacity:
                        pc += curdemand
                        trip.append(n)
                        pnodes.remove(n)

                    if pc == self.capacity:
                        trip.append(0)
                        break

            trips.append(trip)
        self.cal_cost(trips)
        return trips


In [87]:
s = time.time()
a = Rvrp(demand_data,d_data,floor=12,n_robots=2,capacity = 3)
print(a.make_trip())
e = time.time()
print("Total time: ",e-s,"(s)")

Total Travel Time:  3524.1200000000003
[[0, 1, 2, 4, 3, 0, 5, 6, 0], [0, 8, 7, 10, 9, 0, 12, 11, 0]]
Total time:  0.007107973098754883 (s)
